In [19]:
import torch
import torch.nn as nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) # Set for testing purposes, please do not change!


In [2]:
def show_tensor_images(image_tensor, num_images=25, size=(1,28,28)):
    #print images in uniform grid
    image_unflat = image_tensor.detach().cpu.view(-1,*size)
    image_grid = make_grid(image_unflat[:,num_images],mrow=5)
    plt.imshow(image_grid.permute(1,2,0).squeeze())
    plt.show()

In [3]:
def get_generator (input_dim, output_dim ):
    #inmput scalar, output scalar
    #Modules for generator
    return nn.Sequential(nn.Linear(input_dim,output_dim),
                        nn.BatchNorm1d(output_dim),
                        nn.ReLU(inplace=False))


In [24]:
display('a')
class Generator(nn.Module):
    #noise Zdim 
    #in Dim dimension for images mnist(28x28)
    #hidden_dim 
    def __init__(self, z_dim = 10, in_dim = 784, hidden_dim = 128 ):
        super(Generator,self).__init__() #heritage from nn.Module
        self.gen = nn.Sequential(get_generator(z_dim,hidden_dim),
                                get_generator(hidden_dim, hidden_dim*2),
                                get_generator(hidden_dim*2,hidden_dim*4),
                                get_generator(hidden_dim*4,hidden_dim*8),
                                nn.Linear(hidden_dim*8,in_dim),
                                nn.Sigmoid())
    def forward(self, noise):
        return self.gen(noise)
    
    def get_gen(self):
        return self.gen
        
        

'a'

In [17]:
def get_noise (n_samples, z_dim, device = 'cpu'):
    return torch.randn(n_samples, z_dim, device = device)

In [6]:
def get_discriminator(input_dim,output_dim):
    return nn.Sequential(nn.Linear(input_dim,output_dim),
                        nn.LeakyReLU(0.2,inplace=True))

In [ ]:
 class Discriminator(nn.Module):
        def __init__(self, in_dim=784, hidden_dim=128):
            super(Discriminator, self).__init__()
            #Not used Sigmoid since it is included in los function
            self.disc = nn.Sequential(get_discriminator(in_dim,hidden_dim*4),
                                     get_discriminator(hidden_dim*4,hidden_dim*2),
                                     get_discriminator(hidden_dim*2,hidden_dim),
                                     nn.Linear(hidden_dim,1))
        def forward (self,image):
            return self.disc(image)
        
        def get_disc(self):
            return self.disc

In [21]:
display('1')

'1'

In [13]:
criterion = nn.BCEWithLogitsLoss()
n_epochs = 200
z_dim = 64
display_step = 500
batch_size = 128
lr = 0.00001

dataloader = DataLoader(MNIST('.',download = True, transform = transforms.ToTensor()),
                       batch_size = batch_size,
                       shuffle = True)
device = 'cpu'


In [39]:
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(),lr=lr)

disc = Discriminator().to(device)
disc_opt = torch.optim.Adam(disc.parameters(),lr=lr)


In [43]:
def get_disc_loss(gen, disc, criterion, real, num_images, z_dim, device):
    fake_noise = get_noise(num_images, z_dim, device = device)
    fake = gen(fake_noise)#not needed .forward()
    
    disc_fake_pred = disc(fake.detach())
    disc_fake_loss = criterion(disc_fake_pred,torch.zeros_like(disc_fake_pred))
    disc_real_pred = disc(real)
    disc_real_loss = criterion(disc_real_pred,torch.zeros_like(disc_real_pred))
    disc_loss = (disc_fake_loss+disc_real_loss)/2
    return disc_loss
    


In [48]:
def get_gen_loss (gen, disc, criterion, num_images, z_dim, device):
    fake_noise = get_noise(num_images, z_dim, device = device)
    fake = gen(fake_noise)
    
    disc_fake_pred = disc(fake)
    gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
    return gen_loss


In [49]:
#training procedure
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
gen_loss = False
error = False

for epoch in range(n_epochs):
    for real,_ in tqdm(dataloader):
        cur_batch_size = len(real)
        
        real = real.view(cur_batch_size,-1).to(device)
        disc_opt.zero_grad()
        disc_loss = get_disc_loss(gen, disc, criterion, real, cur_batch_size,z_dim, device)
        disc_loss.backward(retain_graph= True)
        disc_opt.step()
        
        gen_opt.zero_grad()
        gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)
        gen_loss.backward()
        gen_opt.step() 
        
        mean_discriminator_loss += disc_loss.item() / display_step
        mean_generator_loss += gen_loss.item() / display_step
        
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Epoch {epoch}, step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            fake = gen(fake_noise)
            show_tensor_images(fake)
            show_tensor_images(real)
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html